In [9]:
import pyfastx

In [3]:
fn = "/ccb/salz4-3/hji20/off-target-probe-checker/data/dev/refseq/GRCh38_latest_assembly_report.txt"

In [4]:
ctg_names = dict()
with open(fn, 'r') as fh:
    for ln in fh:
        if ln[0] == '#': continue
        temp = ln.split("\t")
        # assert len(temp) == 10 # sanity check
        ctg_id = temp[6].strip()
        ctg_name = temp[9].strip()
        ctg_names[ctg_id] = ctg_name

In [5]:
gff_fn = "/ccb/salz4-3/hji20/off-target-probe-checker/data/dev/refseq/GRCh38_latest_genomic.gff"
out_fn = "/ccb/salz4-3/hji20/off-target-probe-checker/data/dev/refseq/refseq.v110.gff"

In [8]:
out_fh = open(out_fn, 'w')
with open(gff_fn, 'r') as fh:
    for ln in fh:
        if ln[0] == '#': continue
        temp = ln.strip().split("\t")
        feature = temp[2]
        if feature == 'region': continue # skip region entries
        if "RNA" in feature or feature == "primary_transcript":
            temp[2] = "transcript"
        ctg_id = temp[0]
        # assert ctg_id in ctg_names # sanity check
        ctg_name = ctg_names[ctg_id]
        temp[0] = ctg_name
        s = '\t'.join(temp)
        out_fh.write(s + "\n")
out_fh.close()

In [10]:
fa = pyfastx.Fasta("/ccb/salz4-3/hji20/off-target-probe-checker/data/dev/refseq/refseq.v110.noAlt.noFix.fa")
tids = set()
for ent in fa:
    tids.add(ent.name)
len(tids)

186819

In [31]:
non_rnas = set()
for tid in tids:
    if tid[0:3] != 'rna':
        if tid[0:4] == 'gene':
            continue
        non_rnas.add(tid)
len(non_rnas)

592

investigate what are these transcripts:

In [33]:
def get_feature_id(s):
    temp = s.split(';')
    fid = None
    for x in temp:
        kv_pair = x.split("=")
        if kv_pair[0] == 'ID':
            fid = kv_pair[1]
    return fid

In [34]:
gff_fn = "/ccb/salz4-3/hji20/off-target-probe-checker/data/dev/refseq/refseq.v110.noAlt.noFix.gff"
ft_tbl = dict()
with open(gff_fn, 'r') as fh:
    for ln in fh:
        temp = ln.strip().split("\t")
        fid = get_feature_id(temp[8])
        if fid in non_rnas:
            if temp[2] in ft_tbl:
                ft_tbl[temp[2]] += 1
            else:
                ft_tbl[temp[2]] = 1
ft_tbl            

{'V_gene_segment': 427,
 'C_gene_segment': 28,
 'J_gene_segment': 105,
 'D_gene_segment': 32}

the decision is to keep these features:

In [35]:
gff_fn = "/ccb/salz4-3/hji20/off-target-probe-checker/data/dev/refseq/refseq.v110.noAlt.noFix.gff"
out_fn = "/ccb/salz4-3/hji20/off-target-probe-checker/data/dev/refseq/refseq.v110.noAlt.noFix.renamed.gff"
out_fh = open(out_fn, 'w')
with open(gff_fn, 'r') as fh:
    for ln in fh:
        temp = ln.strip().split("\t")
        fid = get_feature_id(temp[8])
        if fid in non_rnas:
            temp[2] = "transcript"
        s = '\t'.join(temp)
        out_fh.write(s + "\n")
out_fh.close()

In [37]:
out_fn = "/ccb/salz4-3/hji20/off-target-probe-checker/data/dev/refseq/refseq.v110.noAlt.noFix.filtered.fa"
out_fh = open(out_fn, 'w')
for ent in fa:
    if ent.name[0:3] == 'rna':
        out_fh.write(ent.raw)
    else:
        if ent.name[0:4] == 'gene':
            continue
        out_fh.write(ent.raw)
out_fh.close()